In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.stattools import durbin_watson
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
from scipy.optimize import minimize
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import holidays
import pandas as pd
import numpy as np
import holidays


In [2]:
# Replace 'merged_data.csv' with the exact path to your file if it's not in the same directory
df = pd.read_csv('merged_data.csv')

In [3]:
def transform_and_split_data(df):
    """
    Transform data by pivoting on growth_driver_l5 and splitting periods
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Input dataframe containing:
        - growth_driver_l5 (categories)
        - execution (values)
        - Starting Week
        - Sales Units
        - Average Price
    """
    # Create a copy
    df = df.copy()
    
    # Convert date
    df['Starting Week'] = pd.to_datetime(df['Starting Week'])
    
    # First create the basic pivot
    pivot_exec = df.pivot_table(
        index='Starting Week',
        columns='growth_driver_l5',
        values='execution',
        aggfunc='sum',
        fill_value=0
    ).add_suffix('_execution')
    
    # Now create pre-2023 and post-2022 versions
    pivot_pre2023 = pivot_exec.copy()
    pivot_post2022 = pivot_exec.copy()
    
    # Set values to 0 for respective periods
    pivot_pre2023[pivot_pre2023.index.year >= 2023] = 0
    pivot_post2022[pivot_post2022.index.year < 2023] = 0
    
    # Rename columns to indicate period
    pivot_pre2023.columns = [col + '_pre2023' for col in pivot_pre2023.columns]
    pivot_post2022.columns = [col + '_post2022' for col in pivot_post2022.columns]
    
    # Combine the pivoted data
    result = pd.concat([pivot_pre2023, pivot_post2022], axis=1)
    
    # Add target variable (sum by week)
    weekly_units = df.groupby('Starting Week')["UK L'Oreal Paris Haircare Total Online Sellout Units"].sum()
    
    # Calculate and add average price
    weekly_value = df.groupby('Starting Week')["UK L'Oreal Paris Haircare Total Online Sellout Value (in pound)"].sum()
    weekly_units_for_price = df.groupby('Starting Week')["UK L'Oreal Paris Haircare Total Online Sellout Units"].sum()
    avg_price = (weekly_value / weekly_units_for_price).fillna(0)
    
    # Add to result
    result = result.reset_index()
    result = result.merge(
        pd.DataFrame({
            'Starting Week': weekly_units.index,
            "UK L'Oreal Paris Haircare Total Online Sellout Units": weekly_units.values,
            "UK L'Oreal Paris Haircare Online Average Price (in pound)": avg_price.values
        }),
        on='Starting Week',
        how='left'
    )
    
    return result

In [4]:
result_df = transform_and_split_data(df)
print("Columns in transformed data:")
for col in result_df.columns:
    print(f"- {col}")

Columns in transformed data:
- Starting Week
- amazon_execution_pre2023
- amazon_retail_execution_pre2023
- bvod_execution_pre2023
- citrus_execution_pre2023
- criteo_execution_pre2023
- google_execution_pre2023
- google_video_execution_pre2023
- influencer_management_execution_pre2023
- linear_execution_pre2023
- meta_execution_pre2023
- meta_collab_ads_execution_pre2023
- pinterest_execution_pre2023
- tesco_execution_pre2023
- testers_and_merchandising_execution_pre2023
- the_hut_group_execution_pre2023
- tik_tok_execution_pre2023
- youtube_execution_pre2023
- amazon_execution_post2022
- amazon_retail_execution_post2022
- bvod_execution_post2022
- citrus_execution_post2022
- criteo_execution_post2022
- google_execution_post2022
- google_video_execution_post2022
- influencer_management_execution_post2022
- linear_execution_post2022
- meta_execution_post2022
- meta_collab_ads_execution_post2022
- pinterest_execution_post2022
- tesco_execution_post2022
- testers_and_merchandising_execut

In [5]:
result_df

,Starting Week,amazon_execution_pre2023,amazon_retail_execution_pre2023,bvod_execution_pre2023,citrus_execution_pre2023,criteo_execution_pre2023,google_execution_pre2023,google_video_execution_pre2023,influencer_management_execution_pre2023,linear_execution_pre2023,...,meta_execution_post2022,meta_collab_ads_execution_post2022,pinterest_execution_post2022,tesco_execution_post2022,testers_and_merchandising_execution_post2022,the_hut_group_execution_post2022,tik_tok_execution_post2022,youtube_execution_post2022,UK L'Oreal Paris Haircare Total Online Sellout Units,UK L'Oreal Paris Haircare Online Average Price (in pound)
0,2022-01-03,1468912.9,193246.3,0.00,0.0,156076.7,4067.7,0.0,537.216943,0.00,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,2125879.6,3.242071
1,2022-01-10,2474498.0,343567.9,0.00,0.0,161675.8,3716.7,0.0,0.000000,0.00,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,2457353.6,2.978819
2,2022-01-17,2691219.7,448357.0,0.00,0.0,408499.0,4264.0,0.0,0.000000,0.00,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,2266373.2,3.014039
3,2022-01-24,1940061.5,626307.5,815102.08,0.0,684980.4,4633.2,0.0,142432.121596,58.63,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,2209340.9,3.077403
4,2022-01-31,2349322.3,619993.4,679251.82,0.0,1074118.5,2134.6,0.0,95869.369418,100.49,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,2220604.1,3.306272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,2023-11-27,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.0,0.0,0.0,3038705.800,0.0,0.0,0.0,0.0,2710942.0,3.771592
100,2023-12-04,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.0,0.0,0.0,5438082.650,0.0,0.0,0.0,0.0,2556918.0,3.784533
101,2023-12-11,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.0,0.0,0.0,5438082.650,193.7,0.0,0.0,0.0,2620150.0,3.836532
102,2023-12-18,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.0,0.0,0.0,5438082.650,0.0,0.0,0.0,0.0,2353416.0,3.939211


,Starting Week,amazon_execution_pre2023,amazon_retail_execution_pre2023,bvod_execution_pre2023,citrus_execution_pre2023,criteo_execution_pre2023,google_execution_pre2023,google_video_execution_pre2023,influencer_management_execution_pre2023,linear_execution_pre2023,...,meta_execution_post2022,meta_collab_ads_execution_post2022,pinterest_execution_post2022,tesco_execution_post2022,testers_and_merchandising_execution_post2022,the_hut_group_execution_post2022,tik_tok_execution_post2022,youtube_execution_post2022,UK L'Oreal Paris Haircare Total Online Sellout Units,UK L'Oreal Paris Haircare Online Average Price (in pound)
0,2022-01-03,1468912.9,193246.3,0.00,0.0,156076.7,4067.7,0.0,537.216943,0.00,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,2125879.6,3.242071
1,2022-01-10,2474498.0,343567.9,0.00,0.0,161675.8,3716.7,0.0,0.000000,0.00,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,2457353.6,2.978819
2,2022-01-17,2691219.7,448357.0,0.00,0.0,408499.0,4264.0,0.0,0.000000,0.00,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,2266373.2,3.014039
3,2022-01-24,1940061.5,626307.5,815102.08,0.0,684980.4,4633.2,0.0,142432.121596,58.63,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,2209340.9,3.077403
4,2022-01-31,2349322.3,619993.4,679251.82,0.0,1074118.5,2134.6,0.0,95869.369418,100.49,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,2220604.1,3.306272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,2023-11-27,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.0,0.0,0.0,3038705.800,0.0,0.0,0.0,0.0,2710942.0,3.771592
100,2023-12-04,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.0,0.0,0.0,5438082.650,0.0,0.0,0.0,0.0,2556918.0,3.784533
101,2023-12-11,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.0,0.0,0.0,5438082.650,193.7,0.0,0.0,0.0,2620150.0,3.836532
102,2023-12-18,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.0,0.0,0.0,5438082.650,0.0,0.0,0.0,0.0,2353416.0,3.939211


In [6]:
import pandas as pd
import numpy as np
import holidays

def add_seasonality_features(df, date_col='Starting Week'):
    """Add seasonality features (month/week cyclical, any-day weekend indicator, any-day holiday indicator)."""
    
    df_new = df.copy()
    df_new[date_col] = pd.to_datetime(df_new[date_col])
    
    # Cyclical month
    df_new['month_sin'] = np.sin(2 * np.pi * df_new[date_col].dt.month / 12)
    df_new['month_cos'] = np.cos(2 * np.pi * df_new[date_col].dt.month / 12)
    
    # Cyclical ISO week number
    df_new['week_sin'] = np.sin(
        2 * np.pi * df_new[date_col].dt.isocalendar().week / 52
    )
    df_new['week_cos'] = np.cos(
        2 * np.pi * df_new[date_col].dt.isocalendar().week / 52
    )

    def any_day_in_week_is_holiday(start_date, holiday_calendar):
        for offset in range(7):
            test_date = start_date + pd.Timedelta(days=offset)
            if test_date in holiday_calendar:
                return 1
        return 0

    
    uk_holidays = holidays.country_holidays('GB')
    df_new['is_holiday'] = df_new[date_col].apply(
        lambda d: any_day_in_week_is_holiday(d, uk_holidays)
    )
    
    return df_new


In [7]:
result_df_transformed = add_seasonality_features(result_df, date_col='Starting Week')

In [8]:
result_df_transformed

,Starting Week,amazon_execution_pre2023,amazon_retail_execution_pre2023,bvod_execution_pre2023,citrus_execution_pre2023,criteo_execution_pre2023,google_execution_pre2023,google_video_execution_pre2023,influencer_management_execution_pre2023,linear_execution_pre2023,...,the_hut_group_execution_post2022,tik_tok_execution_post2022,youtube_execution_post2022,UK L'Oreal Paris Haircare Total Online Sellout Units,UK L'Oreal Paris Haircare Online Average Price (in pound),month_sin,month_cos,week_sin,week_cos,is_holiday
0,2022-01-03,1468912.9,193246.3,0.00,0.0,156076.7,4067.7,0.0,537.216943,0.00,...,0.0,0.0,0.0,2125879.6,3.242071,5.000000e-01,0.866025,0.120537,0.992709,1
1,2022-01-10,2474498.0,343567.9,0.00,0.0,161675.8,3716.7,0.0,0.000000,0.00,...,0.0,0.0,0.0,2457353.6,2.978819,5.000000e-01,0.866025,0.239316,0.970942,0
2,2022-01-17,2691219.7,448357.0,0.00,0.0,408499.0,4264.0,0.0,0.000000,0.00,...,0.0,0.0,0.0,2266373.2,3.014039,5.000000e-01,0.866025,0.354605,0.935016,0
3,2022-01-24,1940061.5,626307.5,815102.08,0.0,684980.4,4633.2,0.0,142432.121596,58.63,...,0.0,0.0,0.0,2209340.9,3.077403,5.000000e-01,0.866025,0.464723,0.885456,0
4,2022-01-31,2349322.3,619993.4,679251.82,0.0,1074118.5,2134.6,0.0,95869.369418,100.49,...,0.0,0.0,0.0,2220604.1,3.306272,5.000000e-01,0.866025,0.568065,0.822984,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,2023-11-27,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.0,0.0,0.0,2710942.0,3.771592,-5.000000e-01,0.866025,-0.464723,0.885456,0
100,2023-12-04,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.0,0.0,0.0,2556918.0,3.784533,-2.449294e-16,1.000000,-0.354605,0.935016,0
101,2023-12-11,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.0,0.0,0.0,2620150.0,3.836532,-2.449294e-16,1.000000,-0.239316,0.970942,0
102,2023-12-18,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.0,0.0,0.0,2353416.0,3.939211,-2.449294e-16,1.000000,-0.120537,0.992709,0


,Starting Week,amazon_execution_pre2023,amazon_retail_execution_pre2023,bvod_execution_pre2023,citrus_execution_pre2023,criteo_execution_pre2023,google_execution_pre2023,google_video_execution_pre2023,influencer_management_execution_pre2023,linear_execution_pre2023,...,the_hut_group_execution_post2022,tik_tok_execution_post2022,youtube_execution_post2022,UK L'Oreal Paris Haircare Total Online Sellout Units,UK L'Oreal Paris Haircare Online Average Price (in pound),month_sin,month_cos,week_sin,week_cos,is_holiday
0,2022-01-03,1468912.9,193246.3,0.00,0.0,156076.7,4067.7,0.0,537.216943,0.00,...,0.0,0.0,0.0,2125879.6,3.242071,5.000000e-01,0.866025,0.120537,0.992709,1
1,2022-01-10,2474498.0,343567.9,0.00,0.0,161675.8,3716.7,0.0,0.000000,0.00,...,0.0,0.0,0.0,2457353.6,2.978819,5.000000e-01,0.866025,0.239316,0.970942,0
2,2022-01-17,2691219.7,448357.0,0.00,0.0,408499.0,4264.0,0.0,0.000000,0.00,...,0.0,0.0,0.0,2266373.2,3.014039,5.000000e-01,0.866025,0.354605,0.935016,0
3,2022-01-24,1940061.5,626307.5,815102.08,0.0,684980.4,4633.2,0.0,142432.121596,58.63,...,0.0,0.0,0.0,2209340.9,3.077403,5.000000e-01,0.866025,0.464723,0.885456,0
4,2022-01-31,2349322.3,619993.4,679251.82,0.0,1074118.5,2134.6,0.0,95869.369418,100.49,...,0.0,0.0,0.0,2220604.1,3.306272,5.000000e-01,0.866025,0.568065,0.822984,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,2023-11-27,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.0,0.0,0.0,2710942.0,3.771592,-5.000000e-01,0.866025,-0.464723,0.885456,0
100,2023-12-04,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.0,0.0,0.0,2556918.0,3.784533,-2.449294e-16,1.000000,-0.354605,0.935016,0
101,2023-12-11,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.0,0.0,0.0,2620150.0,3.836532,-2.449294e-16,1.000000,-0.239316,0.970942,0
102,2023-12-18,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.00,...,0.0,0.0,0.0,2353416.0,3.939211,-2.449294e-16,1.000000,-0.120537,0.992709,0


In [18]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.optimize import minimize
from statsmodels.stats.stattools import durbin_watson

# ---------------------------------------------------
# 1) Define your data and variable lists
# ---------------------------------------------------

# Name of your DataFrame:
df = result_df_transformed

# Target variable
target_col = "UK L'Oreal Paris Haircare Total Online Sellout Units"

# Marketing variables (to be forced ≥ 0)
marketing_vars = [
    'amazon_execution_pre2023',
    'amazon_retail_execution_pre2023',
    'bvod_execution_pre2023',
    'citrus_execution_pre2023',
    'criteo_execution_pre2023',
    'google_execution_pre2023',
    'google_video_execution_pre2023',
    'influencer_management_execution_pre2023',
    'linear_execution_pre2023',
    'meta_execution_pre2023',
    'meta_collab_ads_execution_pre2023',
    'pinterest_execution_pre2023',
    'tesco_execution_pre2023',
    'testers_and_merchandising_execution_pre2023',
    'the_hut_group_execution_pre2023',
    'tik_tok_execution_pre2023',
    'youtube_execution_pre2023',
    'amazon_execution_post2022',
    'amazon_retail_execution_post2022',
    'bvod_execution_post2022',
    'citrus_execution_post2022',
    'criteo_execution_post2022',
    'google_execution_post2022',
    'google_video_execution_post2022',
    'influencer_management_execution_post2022',
    'linear_execution_post2022',
    'meta_execution_post2022',
    'meta_collab_ads_execution_post2022',
    'pinterest_execution_post2022',
    'tesco_execution_post2022',
    'testers_and_merchandising_execution_post2022',
    'the_hut_group_execution_post2022',
    'tik_tok_execution_post2022',
    'youtube_execution_post2022',
]

# Seasonality (and other) variables (unconstrained)
seasonality_vars = [
    "UK L'Oreal Paris Haircare Online Average Price (in pound)",
    "month_sin",
    "month_cos",
    "is_holiday"
]

# Create the design matrix X and target y
X = df[marketing_vars + seasonality_vars].copy()
y = df[target_col].values

# Add a constant for the intercept
X = sm.add_constant(X)
# Now X columns will be: ['const'] + marketing_vars + seasonality_vars

# ---------------------------------------------------
# 2) Define the objective function
# ---------------------------------------------------

def objective(params, X, y):
    """Sum of squared residuals (SSE)."""
    residuals = y - X.dot(params)
    return np.sum(residuals**2)

# ---------------------------------------------------
# 3) Set up bounds for each parameter
# ---------------------------------------------------
# - Intercept: no constraint => (None, None)
# - Marketing coefficients: nonnegative => (0.0, None)
# - Seasonality variables: no constraint => (None, None)

n_marketing = len(marketing_vars)
n_seasonality = len(seasonality_vars)

bounds = []
# Intercept (1 parameter)
bounds.append((None, None))
# Marketing variables (≥ 0)
bounds.extend([(0.0, None)] * n_marketing)
# Seasonality variables (unconstrained)
bounds.extend([(None, None)] * n_seasonality)

# ---------------------------------------------------
# 4) Provide an initial guess (from unconstrained OLS)
# ---------------------------------------------------
ols_model = sm.OLS(y, X).fit()
init_params = ols_model.params.values

# ---------------------------------------------------
# 5) Run the constrained optimization
# ---------------------------------------------------
result = minimize(
    objective,
    init_params,
    args=(X, y),
    method='L-BFGS-B',
    bounds=bounds
)

final_params = result.x  # The optimized coefficients

# ---------------------------------------------------
# 6) Evaluate fit: predictions, residuals, R^2, etc.
# ---------------------------------------------------
y_pred = X.dot(final_params)
residuals = y - y_pred

# R^2
ss_res = np.sum(residuals**2)
ss_tot = np.sum((y - np.mean(y))**2)
r2 = 1 - ss_res / ss_tot

# Adjusted R^2
n = len(y)
p = X.shape[1] - 1  # subtract 1 for the intercept
adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

# Durbin-Watson
dw_stat = durbin_watson(residuals)

# ---------------------------------------------------
# 7) Summarize results
# ---------------------------------------------------
param_names = X.columns
coef_dict = dict(zip(param_names, final_params))

print("Constrained OLS Results (nonnegative marketing coefficients):")
print("--------------------------------------------------------------")
for name, val in coef_dict.items():
    print(f"{name:50s} = {val:10.6f}")

print("\nModel Performance:")
print(f"  R-squared           = {r2:.4f}")
print(f"  Adjusted R-squared  = {adj_r2:.4f}")
print(f"  Durbin-Watson       = {dw_stat:.4f}")

if result.success:
    print("\nOptimization converged successfully.")
else:
    print("\nOptimization did NOT converge.")

Constrained OLS Results (nonnegative marketing coefficients):
--------------------------------------------------------------
const                                              = 4092829.023047
amazon_execution_pre2023                           =   0.000000
amazon_retail_execution_pre2023                    =   0.000000
bvod_execution_pre2023                             =   0.000000
citrus_execution_pre2023                           =   0.000000
criteo_execution_pre2023                           =   0.000000
google_execution_pre2023                           =   0.000000
google_video_execution_pre2023                     =   0.218221
influencer_management_execution_pre2023            =   0.094927
linear_execution_pre2023                           =   0.000000
meta_execution_pre2023                             =   0.000000
meta_collab_ads_execution_pre2023                  =   0.011843
pinterest_execution_pre2023                        =   0.000010
tesco_execution_pre2023                